<a href="https://colab.research.google.com/github/17092003vamsi/guner/blob/main/meta_learner_3_L_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load and preprocess data
data = pd.read_csv('project 2 sap.csv')  # Adjust file path if needed
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Lable'].values)
X = data.drop(columns=['Lable'])

# Identify and handle non-numeric columns
# You may need to adapt this based on your specific data and desired handling
for column in X.columns:
    if X[column].dtype == object:  # Check if the column is of object type (likely string)
        try:
            # Attempt to convert to numeric, coercing errors to NaN
            X[column] = pd.to_numeric(X[column], errors='coerce')
        except ValueError:
            # If conversion fails, handle appropriately (e.g., drop, fill with a value, or encode)
            print(f"Column '{column}' contains non-numeric values. Consider dropping, filling, or encoding.")
            # Example: Drop the column
            # X = X.drop(columns=[column])

# Now convert to numpy array after handling non-numeric columns
X = X.values

# Replace NaNs with a fixed value or use imputation
X = np.nan_to_num(X)

# Ensure data is in a supported float type (e.g., float32)
X = X.astype(np.float32)  # Convert to float32

# ... (Rest of your code remains the same) ...

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# CNN Model
def build_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),  # Use Input layer here
        Conv1D(64, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Conv1D(128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Preprocess data for CNN
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val_cnn = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

cnn_model = build_cnn_model((X_train.shape[1], 1))
cnn_model.fit(X_train_cnn, y_train, validation_data=(X_val_cnn, y_val), epochs=20, batch_size=32,
              callbacks=[EarlyStopping(monitor='val_loss', patience=5), ReduceLROnPlateau()])

# Random Forest Model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predictions for Meta-Learner
cnn_val_preds = cnn_model.predict(X_val_cnn).flatten()
rf_val_preds = rf_model.predict_proba(X_val)[:, 1]
meta_X_val = np.vstack((cnn_val_preds, rf_val_preds)).T

cnn_test_preds = cnn_model.predict(X_test_cnn).flatten()
rf_test_preds = rf_model.predict_proba(X_test)[:, 1]
meta_X_test = np.vstack((cnn_test_preds, rf_test_preds)).T

# Meta-Learner Model
meta_model = LogisticRegression()
meta_model.fit(meta_X_val, y_val)

# Final Predictions and Evaluation
meta_test_preds = meta_model.predict(meta_X_test)
print("Meta-Learner Accuracy:", accuracy_score(y_test, meta_test_preds))
print("Classification Report:\n", classification_report(y_test, meta_test_preds))


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 577ms/step - accuracy: 0.2008 - loss: -2.8707 - val_accuracy: 0.2032 - val_loss: 0.5901 - learning_rate: 0.0010
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 580ms/step - accuracy: 0.2023 - loss: -8.3082 - val_accuracy: 0.2086 - val_loss: -4.7231 - learning_rate: 0.0010
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 494ms/step - accuracy: 0.1969 - loss: -12.6303 - val_accuracy: 0.2086 - val_loss: -10.5782 - learning_rate: 0.0010
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 528ms/step - accuracy: 0.2090 - loss: -17.5004 - val_accuracy: 0.2139 - val_loss: -12.6964 - learning_rate: 0.0010
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 599ms/step - accuracy: 0.1768 - loss: -22.5296 - val_accuracy: 0.2406 - val_loss: -18.5986 - learning_rate: 0.0010
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 518ms/step - accuracy: 0.1890 - loss: -27.3282 - val_accuracy: 0.2406 - val_loss: -24.4526 - learning_rate: 0.0010
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 503ms/step - accuracy

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression  # Changed to LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load and preprocess data
data = pd.read_csv('project 2 sap.csv')  # Adjust file path if needed
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Lable'].values)
X = data.drop(columns=['Lable'])

# Identify and handle non-numeric columns
for column in X.columns:
    if X[column].dtype == object:
        try:
            X[column] = pd.to_numeric(X[column], errors='coerce')
        except ValueError:
            print(f"Column '{column}' contains non-numeric values. Consider dropping, filling, or encoding.")

X = X.values
X = np.nan_to_num(X)
X = X.astype(np.float32)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# CNN Model
def build_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(64, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Conv1D(128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Preprocess data for CNN
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val_cnn = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

cnn_model = build_cnn_model((X_train.shape[1], 1))
cnn_model.fit(X_train_cnn, y_train, validation_data=(X_val_cnn, y_val), epochs=20, batch_size=32,
              callbacks=[EarlyStopping(monitor='val_loss', patience=5), ReduceLROnPlateau()])

# Evaluate CNN model
cnn_test_preds = (cnn_model.predict(X_test_cnn) > 0.5).astype("int32").flatten()
cnn_accuracy = accuracy_score(y_test, cnn_test_preds)
print("CNN Model Accuracy:", cnn_accuracy)
print("CNN Classification Report:\n", classification_report(y_test, cnn_test_preds))

# Random Forest Model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate Random Forest model
rf_test_preds = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_test_preds)
print("Random Forest Model Accuracy:", rf_accuracy)
print("Random Forest Classification Report:\n", classification_report(y_test, rf_test_preds))

# Predictions for Meta-Learner
cnn_val_preds = cnn_model.predict(X_val_cnn).flatten()
rf_val_preds = rf_model.predict_proba(X_val)[:, 1]
meta_X_val = np.vstack((cnn_val_preds, rf_val_preds)).T

cnn_test_preds_meta = cnn_model.predict(X_test_cnn).flatten()
rf_test_preds_meta = rf_model.predict_proba(X_test)[:, 1]
meta_X_test = np.vstack((cnn_test_preds_meta, rf_test_preds_meta)).T

# Meta-Learner Model with Linear Regression
meta_model = LinearRegression()
meta_model.fit(meta_X_val, y_val)

# Make predictions with the meta-learner and convert them to binary (0 or 1)
meta_test_preds = (meta_model.predict(meta_X_test) > 0.5).astype(int)

# Evaluate Meta-Learner model
meta_accuracy = accuracy_score(y_test, meta_test_preds)
print("Meta-Learner (Linear Regression) Accuracy:", meta_accuracy)
print("Meta-Learner Classification Report:\n", classification_report(y_test, meta_test_preds))


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 622ms/step - accuracy: 0.2002 - loss: -2.6186 - val_accuracy: 0.2032 - val_loss: 8.3624 - learning_rate: 0.0010
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 975ms/step - accuracy: 0.2168 - loss: -7.8799 - val_accuracy: 0.2086 - val_loss: -5.9010 - learning_rate: 0.0010
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 745ms/step - accuracy: 0.2180 - loss: -13.6955 - val_accuracy: 0.2086 - val_loss: -8.3001 - learning_rate: 0.0010
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 600ms/step - accuracy: 0.2043 - loss: -17.4865 - val_accuracy: 0.2139 - val_loss: -12.6827 - learning_rate: 0.0010
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 486ms/step - accuracy: 0.1877 - loss: -21.6416 - val_accuracy: 0.2246 - val_loss: -15.9097 - learning_rate: 0.0010
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 562ms/step - accuracy: 0.1770 - loss: -27.2295 - val_accuracy: 0.2246 - val_loss: -20.6384 - learning_rate: 0.0010
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 592ms/step - accuracy:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Random Forest Model Accuracy: 0.9411764705882353
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98        31
           1       0.97      0.94      0.95        32
           2       0.89      0.94      0.92        36
           3       1.00      0.96      0.98        27
           4       0.92      0.83      0.87        29
           5       0.91      0.97      0.94        32

    accuracy                           0.94       187
   macro avg       0.94      0.94      0.94       187
weighted avg       0.94      0.94      0.94       187

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Meta-Learner (Linear Regression) Accuracy: 0.26737967914438504
Meta-Learner Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.77      0.79        31
           1       0.17      0.81      0.28        32
           2       0.00      0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
